# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [450]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

### 데이터 셋 읽어오기

In [451]:
df = pd.read_csv("train.csv",  encoding='ISO-8859-1') # 학습용 데이터

In [452]:
df.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


## 2. 데이터 전처리

### NA 데이터 처리

In [453]:
#1. is_converted가 NA인 데이터 drop
original_rows = len(df)
df = df.dropna(subset=['is_converted'])
erased_rows = original_rows - len(df)
print('지워진 행: '+str(erased_rows))
print('is_converted unique values:')
df.is_converted.unique()

지워진 행: 0
is_converted unique values:


array([ True, False])

In [454]:
# is_converted 의 값들 정리
# 모두 TRUE or FALSE로 통일, 이외값을 가진 행은 제거
df['is_converted'] = df['is_converted'].astype(str).str.upper()
df = df[df['is_converted'].isin(['TRUE', 'FALSE'])]
print('지워진 행: '+ str(original_rows-erased_rows-len(df)))

지워진 행: 0


In [455]:
# 중복 행 처리
df.drop_duplicates(keep='first', inplace=True)
len(df)

55780

In [456]:
# 아래 3개 칼럼들은 가중치 값이라는 도메인지식이 있음. 
# 원본 파일을 확인하면 가중치값이 1밖에 없다는것을 확인 가능한데, 
# 1이라는 가중치의 의미는 숫자로서의 의미보다는, 
# 존재여부, 유무여부정도로 해석되어야 할것으로 보임. 
# 하여 NA들은 모두 0으로 채워주기로 함.

# 추가적으로, 해당 가중치들은 고객들이 채우는것이 아니라, LG영업팀이 
# 주는것으로 보아, NA는 

df.id_strategic_ver.fillna(0, inplace=True)
df.it_strategic_ver.fillna(0, inplace=True)
df.idit_strategic_ver.fillna(0, inplace=True)

print(df.id_strategic_ver.unique())
print(df.it_strategic_ver.unique())
print(df.idit_strategic_ver.unique())

[0. 1.]
[0. 1.]
[0. 1.]


In [457]:
#business_subarea 칼럼값은 고객이 주지 않은 정보이므로 NA로 처리

df.business_subarea.fillna('NA', inplace=True)
print(len(df.business_subarea.unique()))

87


In [458]:
#칼럼별 NA율이 50% 이상인 칼럼 나열
for col in df.columns:
    if df[col].isnull().sum()/len(df)*100 > 50:
        print(col, df[col].isnull().sum()/len(df)*100)

com_reg_ver_win_rate 74.69702402294729
customer_type 74.14485478666188
historical_existing_cnt 77.82359268555038
product_subcategory 83.85442811043384
product_modelname 83.80960917891717
expected_timeline 50.72068841878809
ver_win_rate_x 68.11222660451774
ver_win_ratio_per_bu 73.31660093223378
business_area 68.11222660451774


In [459]:
# com_reg_ver_win_rate 
# 해당 칼럼 역시 LG 가 고객 정보를 토대로 직접 만드는 정보임.
# business_unit 과 region에 따라서 주는 oppty rate 계산인데, 
# numerical data 이므로, NA값들은 knn imputation을 통해 채워주기로 함.

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df['com_reg_ver_win_rate'] = pd.to_numeric(df['com_reg_ver_win_rate'], errors='coerce')

df['com_reg_ver_win_rate'] = imputer.fit_transform(df[['com_reg_ver_win_rate']])
df['com_reg_ver_win_rate'].head()

0    0.066667
1    0.066667
2    0.088889
3    0.088889
4    0.088889
Name: com_reg_ver_win_rate, dtype: float64

In [460]:
# customer_type
# 고객이 넣어줘야하는 정보이므로, NA값들은 Unknown으로 처리

df['customer_type'] = df['customer_type'].fillna('NA')

In [461]:
# historical_existing_cnt
#도메인 지식으로는, 이 데이터가 회사에서 온것이므로, 회사가 단 한번이라도 고객을 상대했더라면 그 기록이 남았을것이다. 
# 따라서 이 칼럼의 NA는 전적이 없다, 즉 0으로 채울 수 있다. Domain Knowledge에 의한 imputation

# df = df.drop(columns=['historical_existing_cnt'])

df.historical_existing_cnt.fillna(0, inplace=True)
df.historical_existing_cnt.unique()

array([0.000e+00, 1.200e+01, 1.440e+02, 3.000e+00, 2.300e+01, 4.700e+01,
       1.000e+00, 1.900e+01, 4.200e+01, 7.500e+01, 1.700e+01, 2.000e+01,
       5.000e+00, 2.000e+00, 1.150e+02, 4.000e+00, 1.600e+01, 6.000e+00,
       3.100e+01, 4.900e+01, 1.000e+01, 1.100e+01, 4.000e+01, 7.000e+00,
       6.100e+01, 4.500e+01, 5.700e+01, 1.750e+02, 1.300e+01, 9.900e+01,
       1.500e+01, 8.000e+00, 2.100e+01, 5.600e+01, 6.660e+02, 1.220e+02,
       3.240e+02, 2.200e+01, 8.000e+01, 3.600e+01, 2.800e+01, 1.720e+02,
       6.600e+01, 3.200e+01, 5.300e+01, 3.800e+01, 1.890e+02, 5.800e+01,
       2.700e+01, 1.060e+02, 1.680e+02, 9.000e+00, 4.600e+01, 4.800e+01,
       5.200e+01, 7.300e+01, 9.200e+01, 6.000e+01, 3.900e+01, 1.800e+01,
       1.400e+01, 2.120e+02, 4.720e+02, 3.900e+02, 6.900e+01, 2.330e+02,
       2.900e+01, 3.000e+01, 2.400e+01, 7.800e+01, 1.070e+02, 4.100e+01,
       1.660e+02, 5.900e+01, 4.400e+01, 1.140e+02, 3.700e+01, 3.500e+01,
       1.580e+02, 8.500e+01, 7.700e+01, 2.600e+01, 

In [462]:
# product_subcategory
# 위와 같은 이유, 고객이 넣어줘야하는 정보이므로, NA값들은 Not Answered 의미로 처리
df['product_subcategory'] = df['product_subcategory'].fillna('NA')

In [463]:
# product_modelname
# 위와 동일한 이유
df['product_modelname'] = df['product_modelname'].fillna('NA')

In [464]:
# expected_timeline
# 많이 raw한 데이터 정제

# Replace the specified 'expected_timeline' values
df['expected_timeline'] = df['expected_timeline'].replace({
    'less_than_3_months': 'less than 3 months',  # merge into 'less than 3 months'
    'being followed up': 'being followed up.',  # merge into 'being followed up.'
    'less than 6 months': '3 months ~ 6 months',  # merge into '3 months ~ 6 months'
    '3_months_~_6_months' : '3 months ~ 6 months'
})

top_expected_timelines = df['expected_timeline'].value_counts().nlargest(6).index

# Replace all other 'expected_timeline' values with 'Others'
df['expected_timeline'] = df['expected_timeline'].apply(lambda x: x if x in top_expected_timelines else 'Others')

# Optional: Check the updated value counts to confirm the merges
updated_counts = df['expected_timeline'].value_counts()
print(updated_counts)


expected_timeline
Others                 28940
less than 3 months     16843
3 months ~ 6 months     4915
more than a year        2874
9 months ~ 1 year       1070
6 months ~ 9 months     1043
etc.                      95
Name: count, dtype: int64


In [465]:
df['expected_timeline'] = df['expected_timeline'].fillna('etc')

In [466]:
# ver_win_rate_x
# numerical data로 knn imputation을 통해 NA값들을 채워줌

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

df['ver_win_rate_x'] = pd.to_numeric(df['ver_win_rate_x'], errors='coerce')

df['ver_win_rate_x'] = imputer.fit_transform(df[['ver_win_rate_x']])

df['ver_win_rate_x'].head()

0    0.003079
1    0.003079
2    0.003079
3    0.003079
4    0.003079
Name: ver_win_rate_x, dtype: float64

In [467]:
# ver_win_ratio_per_bu
# 위와 같이 numerical data로 knn imputation을 통해 NA값들을 채워줌

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

df['ver_win_ratio_per_bu'] = pd.to_numeric(df['ver_win_ratio_per_bu'], errors='coerce')
df['ver_win_ratio_per_bu'] = imputer.fit_transform(df[['ver_win_ratio_per_bu']])
df['ver_win_ratio_per_bu'].head()

0    0.026846
1    0.026846
2    0.026846
3    0.026846
4    0.026846
Name: ver_win_ratio_per_bu, dtype: float64

In [468]:
# business_area
# 사업부는 고객이 넣어줘야하는 정보이므로, NA값들은 Not Answered 의미로 처리
df['business_area'] = df['business_area'].fillna('NA')

In [469]:
# customer_job
# 고객 직업은 고객이 넣어줘야하는 정보이므로, NA값들은 Not Answered 의미로 처리
# 의사결정자이냐 아니냐는 부분이 중요함
df['customer_job'] = df['customer_job'].fillna('NA')

In [470]:
# product_category
# 위 product_subcategory와 같은 이유
df['product_category'] = df['product_category'].fillna('NA')

In [471]:
# customer_country.1

def extract_country(value, response_corporate):
    # Mapping for response_corporate to country names
    corporate_to_country = {
        'LGEUS': 'United States',
        'LGECB': 'Colombia',
        'LGEDG': 'Germany',
        'LGEIL': 'India',
        'LGESP': 'Brazil',
        'LGEPH': 'Philippines',
        'LGEIN': 'Indonesia',
        'LGEEG': 'Egypt',
        'LGEUK': 'United Kingdom',
        'LGETK': 'Turkey',
        'LGEMS': 'Mexico',
        'LGEFS': 'France',
        'LGEES': 'Spain'
    }
    
    # Function to determine if replacement is needed based on the content of the value
    def needs_replacement(val):
        if '@' in val or any(char.isdigit() for char in val):
            return True
        words = val.split()
        return len(words) > 2
    
    # Initial check for NaN or non-string types
    if pd.isna(value) or not isinstance(value, str):
        return 'NA'
    
    # Check for email formats or if replacement is needed based on the content
    if '@' in value or needs_replacement(value):
        return 'Egypt' if '@' in value else corporate_to_country.get(response_corporate, 'NA')
    
    # Modified process to handle '//'
    country = 'NA'
    if '//' in value:
        # Split the value on '//' and take the last part
        parts = value.split('//')
        country = parts[-1].strip().title() if parts[-1].strip() else parts[-2].strip().title()
    elif '/' in value:
        parts = value.strip('/').split('/')
        country = parts[-1].strip().title()
    
    # Final check for replacement based on processed country
    if needs_replacement(country):
        return corporate_to_country.get(response_corporate, 'NA')
    
    if country == ' ':
        return corporate_to_country.get(response_corporate, 'NA')
    
    return country if country != 'NA' else value.strip().title()


# Re-apply the function with the enhanced logic
df['customer_country.1_normalized'] = df.apply(lambda x: extract_country(x['customer_country.1'], x['response_corporate']), axis=1)

# Verify the transformation
print(df[['customer_country.1', 'response_corporate', 'customer_country.1_normalized']].head())


         customer_country.1 response_corporate customer_country.1_normalized
0  /Quezon City/Philippines              LGEPH                   Philippines
1        /PH-00/Philippines              LGEPH                   Philippines
2           /Kolkata /India              LGEIL                         India
3        /Bhubaneswar/India              LGEIL                         India
4          /Hyderabad/India              LGEIL                         India


In [472]:
df['customer_country.1'] = df['customer_country.1_normalized']
df.drop(columns=['customer_country.1_normalized'], inplace=True)
len(df.columns)

29

In [473]:
df['customer_country.1'].fillna('NA', inplace=True)

#count NA values in customer_country.1
df['customer_country.1'].isnull().sum()

0

In [474]:
#customer_country

# Apply the function to your DataFrame
df['customer_country_normalized_a'] = df.apply(lambda x: extract_country(x['customer_country'], x['response_corporate']), axis=1)

# Verify the transformation and cleanup
print(df[['customer_country', 'response_corporate', 'customer_country_normalized_a']].head())


df['customer_country'] = df['customer_country_normalized_a']
df.drop(columns=['customer_country_normalized_a'], inplace=True)
len(df.columns)

           customer_country response_corporate customer_country_normalized_a
0  /Quezon City/Philippines              LGEPH                   Philippines
1        /PH-00/Philippines              LGEPH                   Philippines
2           /Kolkata /India              LGEIL                         India
3        /Bhubaneswar/India              LGEIL                         India
4          /Hyderabad/India              LGEIL                         India


29

In [475]:
df['customer_country'].fillna('NA', inplace=True)

#count NA values in customer_country.1
df['customer_country'].isnull().sum()

0

In [476]:
# inquiry_type
# df['inquiry_type'] = df['inquiry_type'].fillna('NA')

print(len(df['inquiry_type'].unique()))
df['inquiry_type'].value_counts()

# Replace the specified 'inquiry_type' values
df['inquiry_type'] = df['inquiry_type'].replace({
    'Quotation or Purchase Consultation': 'Quotation or purchase consultation'
})

72


In [477]:
# First, calculate the value counts for each unique value in 'inquiry_type'
value_counts = df['inquiry_type'].value_counts()

# Identify those unique values where the count is less than 900
values_to_replace = value_counts[value_counts < 900].index.tolist()

# Replace those values with 'Other'
df['inquiry_type'] = df['inquiry_type'].replace(values_to_replace, 'Other')

# Your specified manual replacement can also be included before or after this operation
df['inquiry_type'] = df['inquiry_type'].replace({
    'Quotation or Purchase Consultation': 'Quotation or purchase consultation',
    # Add any other specific replacements here
})

df['inquiry_type'] = df['inquiry_type'].fillna('NA')

print(len(df['inquiry_type'].unique()))
df['inquiry_type'].value_counts()

5


inquiry_type
Quotation or purchase consultation    40136
Sales Inquiry                          8844
Other                                  4809
Product Information                    1100
NA                                      891
Name: count, dtype: int64

### 2-2. 카테고리 데이터 인코딩

In [478]:
# categorical columns

for col in df.columns:
    if df[col].dtype != 'int64' and df[col].dtype != 'float64':
        print(col)

customer_country
business_unit
customer_type
enterprise
customer_job
inquiry_type
product_category
product_subcategory
product_modelname
customer_country.1
customer_position
response_corporate
expected_timeline
business_area
business_subarea
is_converted


In [479]:
df['is_converted'].unique()

array(['TRUE', 'FALSE'], dtype=object)

### business_unit

In [482]:
df.business_unit.unique()

array(['AS', 'ID', 'IT', 'Solution', 'CM'], dtype=object)

In [484]:
# Apply one-hot encoding to 'business_unit'
from sklearn.model_selection import KFold
# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'business_area'
df['business_unit_te'] = np.nan

for train_idx, val_idx in kf.split(df):
    # Split the data into training and validation sets
    X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
    
    # Compute the target mean for each category in 'business_area' on the training data
    means = X_train.groupby('business_unit')['is_converted'].mean()
    
    # Map the computed means to the validation set
    df.loc[val_idx, 'business_unit_te'] = X_val['business_unit'].map(means)

# Fill missing values for any 'business_area' not seen in the training folds with the global mean
global_mean = df['is_converted'].mean()
df['business_unit_te'].fillna(global_mean, inplace=True)

# Verify the changes
print(df[['business_unit', 'business_unit_te']].head())

KeyError: '[322, 344, 1253, 1314, 1477, 1487, 1489, 1497, 1503, 1684, 2829, 3279, 3418, 3422, 3615, 3986, 5701, 5990, 6247, 6423, 6561, 7284, 7567, 8344, 10331, 10688, 10791, 11076, 11077, 11081, 11092, 11097, 11107, 11113, 11150, 11165, 11168, 11178, 11186, 11200, 11203, 11208, 11212, 11235, 11245, 11247, 11250, 11461, 12428, 12472, 12496, 12506, 12520, 12551, 12639, 12701, 12792, 12796, 12909, 13013, 13035, 13039, 13059, 13124, 13131, 13167, 13249, 13253, 13270, 13297, 13298, 13301, 13413, 14233, 14238, 14608, 14636, 15130, 15292, 15602, 15603, 15625, 15632, 15639, 15643, 15650, 15712, 15738, 15742, 16174, 16599, 16718, 16814, 16886, 16927, 17418, 17811, 18176, 18180, 18210, 18219, 18222, 18224, 18352, 18397, 18618, 18629, 18641, 18848, 18874, 18960, 18968, 19035, 19166, 19176, 19209, 19231, 19315, 19319, 19327, 19460, 19740, 19860, 19967, 20071, 20073, 20155, 20164, 20170, 20184, 20196, 20330, 20339, 20367, 20443, 20839, 20847, 20853, 20861, 20865, 20891, 20892, 20904, 20909, 20937,

In [485]:
# Example indices from your error message
indices_to_check = [322, 344, 1253, 55756]  # Add more as needed

# Check each index and access value if present
for idx in indices_to_check:
    if idx in df.index:
        # Accessing a specific column for these indices, assuming 'business_unit' as an example
        value = df.loc[idx, 'business_unit'] if 'business_unit' in df.columns else 'Column Not Found'
        print(f"Index {idx}: value = {value}")
    else:
        print(f"Index {idx} is not present in the DataFrame.")


Index 322 is not present in the DataFrame.
Index 344 is not present in the DataFrame.
Index 1253 is not present in the DataFrame.
Index 55756 is not present in the DataFrame.


In [486]:
df.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,customer_country_te,business_unit_te
0,1.0,Philippines,AS,0.066667,32160,End-Customer,Enterprise,0.0,0.0,0.0,...,1,0,0.003079,0.026846,corporate / office,Engineering,0,1,NaN,NaN
1,1.0,Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,0.0,0.0,...,1,0,0.003079,0.026846,corporate / office,Advertising,1,1,NaN,NaN
2,1.0,India,AS,0.088889,1755,End-Customer,Enterprise,144.0,0.0,0.0,...,1,0,0.003079,0.026846,corporate / office,Construction,2,1,NaN,NaN
3,1.0,India,AS,0.088889,4919,End-Customer,Enterprise,0.0,0.0,0.0,...,1,0,0.003079,0.026846,corporate / office,IT/Software,3,1,NaN,NaN
4,1.0,India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,0.0,0.0,0.0,...,0,0,0.003079,0.026846,corporate / office,NA,4,1,NaN,NaN


In [487]:
df.to_csv('current_train.csv', index=False)

### customer_country

In [480]:
print(len(df['customer_country'].unique()))

287


In [481]:
# Convert 'is_converted' from 'TRUE'/'FALSE' to integers (1/0)
df['is_converted'] = df['is_converted'].map({'TRUE': 1, 'FALSE': 0})

# Prepare the StratifiedKFold object
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Placeholder for the target encoded feature
df['customer_country_te'] = np.nan



# Assuming the rest of your setup is correct, modify the mapping part like this:
for train_index, test_index in skf.split(df, df['is_converted']):
    # Your existing setup...
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]

    # Compute target mean for each category in 'customer_country'
    means = X_train.groupby('customer_country')['is_converted'].mean()
    
    # Use a safer mapping approach to avoid KeyError
    df.loc[test_index, 'customer_country_te'] = X_test['customer_country'].apply(lambda x: means.get(x, np.nan))

# Then, handle any remaining NaN values in 'customer_country_te'
global_mean = df['is_converted'].mean()
df['customer_country_te'].fillna(global_mean, inplace=True)

# for train_index, test_index in skf.split(df, df['is_converted']):
#     # Split the data
#     X_train, X_test = df.iloc[train_index], df.iloc[test_index]

#     assert 'customer_country' in X_test.columns, "'customer_country' column is missing in X_test."
#     assert test_index.max() < len(df), "Some indices in test_index are out of bounds for df."

#     # Compute target mean for each category in 'customer_country' excluding the current fold
#     means = X_train.groupby('customer_country')['is_converted'].mean()
    
#     # Map the computed means to the test portion of the fold
#     df.loc[test_index, 'customer_country_te'] = X_test['customer_country'].map(means)

# # For any missing values in the target encoded feature (e.g., categories present only in test folds),
# # fill with the global mean of 'is_converted'
# global_mean = df['is_converted'].mean()
# df['customer_country_te'].fillna(global_mean, inplace=True)

# Show the first few rows to verify
print(df[['customer_country', 'customer_country_te']].head())

KeyError: '[1341, 1401, 1437, 2589, 3012, 5690, 6382, 7567, 8347, 9260, 10568, 11083, 11089, 11094, 11115, 11136, 11139, 11150, 11151, 11168, 11180, 11183, 11193, 11228, 11231, 11242, 11951, 12057, 12379, 12496, 12553, 12641, 12721, 13013, 13043, 13124, 13147, 13179, 13234, 13270, 13297, 13733, 14483, 14566, 14607, 14758, 15241, 15623, 15638, 15714, 16886, 17038, 18229, 18643, 18649, 18807, 18890, 18892, 19095, 19217, 19275, 19313, 19319, 19764, 19860, 20083, 20367, 20443, 20445, 20453, 20793, 20918, 20923, 21053, 21172, 21186, 21204, 21254, 21326, 21490, 21491, 21502, 21515, 21526, 21536, 21547, 21556, 21560, 21568, 21581, 21595, 21615, 22221, 22267, 22365, 23476, 23489, 23519, 23531, 23576, 23603, 23855, 24016, 24036, 24122, 24133, 24135, 24646, 25648, 26334, 26353, 26460, 26649, 26854, 27354, 27382, 29299, 30125, 31109, 31143, 31399, 31433, 31434, 32578, 32765, 33711, 34073, 34853, 35137, 35390, 35415, 35460, 35467, 36070, 36717, 36921, 37720, 37921, 37957, 38846, 38905, 38966, 3902

### customer_country.1

In [ ]:
len(df['customer_country.1'].unique())

425

In [ ]:
# Convert 'is_converted' from 'TRUE'/'FALSE' to integers (1/0)
df['is_converted'] = df['is_converted'].map({'TRUE': 1, 'FALSE': 0})

# Prepare the StratifiedKFold object
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Placeholder for the target encoded feature
df['customer_country.1_te'] = np.nan

for train_index, test_index in skf.split(df, df['is_converted']):
    # Split the data
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    
    # Compute target mean for each category in 'customer_country.1' excluding the current fold
    means = X_train.groupby('customer_country.1')['is_converted'].mean()
    
    # Map the computed means to the test portion of the fold
    df.loc[test_index, 'customer_country.1_te'] = X_test['customer_country.1'].map(means)

# For any missing values in the target encoded feature (e.g., categories present only in test folds),
# fill with the global mean of 'is_converted'
global_mean = df['is_converted'].mean()
df['customer_country.1_te'].fillna(global_mean, inplace=True)

# Show the first few rows to verify
print(df[['customer_country.1', 'customer_country.1_te']].head())

KeyError: '[1341, 1401, 1437, 2589, 3012, 5690, 6382, 7567, 8347, 9260, 10568, 11083, 11089, 11094, 11115, 11136, 11139, 11150, 11151, 11168, 11180, 11183, 11193, 11228, 11231, 11242, 11951, 12057, 12379, 12496, 12553, 12641, 12721, 13013, 13043, 13124, 13147, 13179, 13234, 13270, 13297, 13733, 14483, 14566, 14607, 14758, 15241, 15623, 15638, 15714, 16886, 17038, 18229, 18643, 18649, 18807, 18890, 18892, 19095, 19217, 19275, 19313, 19319, 19764, 19860, 20083, 20367, 20443, 20445, 20453, 20793, 20918, 20923, 21053, 21172, 21186, 21204, 21254, 21326, 21490, 21491, 21502, 21515, 21526, 21536, 21547, 21556, 21560, 21568, 21581, 21595, 21615, 22221, 22267, 22365, 23476, 23489, 23519, 23531, 23576, 23603, 23855, 24016, 24036, 24122, 24133, 24135, 24646, 25648, 26334, 26353, 26460, 26649, 26854, 27354, 27382, 29299, 30125, 31109, 31143, 31399, 31433, 31434, 32578, 32765, 33711, 34073, 34853, 35137, 35390, 35415, 35460, 35467, 36070, 36717, 36921, 37720, 37921, 37957, 38846, 38905, 38966, 3902

In [ ]:
# replace the original column with the target encoded column, but the order matters

df['customer_country.1'] = df['customer_country.1_te']
df.drop('customer_country.1_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

### business_unit

In [ ]:
df.business_unit.unique()

In [ ]:
# Apply one-hot encoding to 'business_unit'
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'business_area'
df['business_unit_te'] = np.nan

for train_idx, val_idx in skf.split(df):
    # Split the data into training and validation sets
    X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
    
    # Compute the target mean for each category in 'business_area' on the training data
    means = X_train.groupby('business_unit')['is_converted'].mean()
    
    # Map the computed means to the validation set
    df.loc[val_idx, 'business_unit_te'] = X_val['business_unit'].map(means)

# Fill missing values for any 'business_area' not seen in the training folds with the global mean
global_mean = df['is_converted'].mean()
df['business_unit_te'].fillna(global_mean, inplace=True)

# Verify the changes
print(df[['business_unit', 'business_unit_te']].head())

In [ ]:
df['business_unit'] = df['business_unit_te']
df.drop('business_unit_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

### customer_type

In [ ]:
print(df.customer_type.unique())
len(df.customer_type.unique())

In [ ]:
# Initialize KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'customer_type'
df['customer_type_te'] = np.nan

# Apply K-fold target encoding
for train_idx, val_idx in skf.split(df):
    # Split the data
    df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]
    # Compute the target mean for each category in 'customer_type' on the training data
    means = df_train.groupby('customer_type')['is_converted'].mean()
    # Map the computed means to the validation set
    df.loc[val_idx, 'customer_type_te'] = df_val['customer_type'].map(means)

# Fill missing values for any 'customer_type' not seen in the training folds
global_mean = df['is_converted'].mean()
df['customer_type_te'].fillna(global_mean, inplace=True)

### 2-3. 학습, 검증 데이터 분리

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    df.drop("is_converted", axis=1),
    df["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

## 3. 모델 학습

### 모델 정의 

In [ ]:
model = DecisionTreeClassifier()

### 모델 학습

In [ ]:
model.fit(x_train, y_train)

ValueError: could not convert string to float: 'India'

### 모델 성능 보기

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[ 715  213]
 [ 248 9980]]

정확도: 0.9587
정밀도: 0.7425
재현율: 0.7705
F1: 0.7562


## 4. 제출하기

### 테스트 데이터 예측

In [ ]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [ ]:
test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- subcat_55" 500 nits FHD 0.44mm Even Bezel Video Wall
- subcat_55" 700 nits FHD 0.44mm Even Bezel Video Wall
- subcat_55EF5F-L
- subcat_55EW5TK-A
- subcat_55VH7J-H
- ...
Feature names seen at fit time, yet now missing:
- business_unit_CM
- business_unit_Solution
- subcat_28MQ780
- subcat_43HT3WJ
- subcat_49" 500 nits FHD Slim Bezel Video Wall
- ...


### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("sub.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**